In [ ]:
# import
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torchvision.datasets import CIFAR10, CIFAR100
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

In [ ]:
# create checkpoint path, if not exists
if not os.path.exists('checkpoint'):
    os.mkdir('checkpoint')

In [ ]:
# set batch-sizes and torch device
BATCH_SIZE = 64
TRIGGER_BATCH_SIZE = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# create a derived dataset class to use in dataloader
class TriggerSet(Dataset):
    def __init__(self, trigger_path, transform=None):
        self.images = []
        self.labels = []
        labels_folder = os.path.join(trigger_path, "labels")
        images_folder = os.path.join(trigger_path, "images")
        with open(os.path.join(labels_folder, "trigger_labels.txt"), "r") as file:
            for line in file:
                label = int(line.strip())
                self.labels.append(label)
        for i in range(len(self.labels)):
            img_name = f"{i}.jpg"
            img_path = os.path.join(images_folder, img_name)
            img = Image.open(img_path).convert("RGB")
            if transform:
                img = transform(img)
            self.images.append(img)

    def __getitem__(self, index):
        return self.images[index], self.labels[index]

    def __len__(self):
        return len(self.labels)

In [ ]:
def evaluate(model, dataloader):
    """
    A function to evaluate given model, on a given data inside a dataloder.
    Input:
        model = pytorch trained model
        dataloader = a dataloader object with test data
    Output:
        an integer, accuracy of the model.
    """
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()
    return (100 * correct) / total

def save_checkpoint(epoch, model, optimizer, path):
    """
    A function to save given model's, and optimizer's state.
    Input:
        epoch = an integer, trained number of epochs
        optimizer = optimizer object with current status, learnin rate etc.
        model = pytorch model with current learnable parameters
        path = a string indicating the path of checkpoint
    Output:
        No output, save the to given path.
    """
    checkpoint_name = f"checkpoint_{epoch}.pt"
    checkpoint_path = os.path.join(path, checkpoint_name)
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, checkpoint_path)

def load_checkpoint(model, optimizer, path):
    """
    A function to load given model's, and optimizer's state.
    Input:
        optimizer =  optimizer object to be updated
        model =  pytorch model to be updated
        path = model path to be loaded
    Output:
        No output, save the to given path.
    """
    checkpoint = torch.load(path)
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return epoch

In [ ]:
# transform the images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((640, 640))
])

In [ ]:
# set training environment
use_trigger = False
dataset = "cifar10"
trigger_path = "/kaggle/input/trigger/data/trigger_set"

In [ ]:
# 46k-4k training-test split from CIFAR10/CIFAR100-training data
if dataset == "cifar10":
    num_classes = 10
    full_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainset = torch.utils.data.Subset(full_dataset, range(46000))
    testset = torch.utils.data.Subset(full_dataset, range(46000, 50000))
elif dataset == "cifar100":
    num_classes = 100
    full_dataset = CIFAR100(root='./data', train=True, download=True, transform=transform)
    trainset = torch.utils.data.Subset(full_dataset, range(46000))
    testset = torch.utils.data.Subset(full_dataset, range(46000, 50000))

In [ ]:
# crate trigger data set
trigger_set = TriggerSet(trigger_path, transform=transform)

dataloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_dataloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
trigger_dataloader = DataLoader(trigger_set, batch_size=TRIGGER_BATCH_SIZE, shuffle=True, num_workers=2)

model = resnet18(num_classes=num_classes).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

In [1]:
# train
for epoch in range(30):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader))

    for batch_idx, (images, labels) in pbar:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        if use_trigger:
            trigger_images, trigger_labels = next(iter(trigger_dataloader))
            trigger_images = trigger_images.to(DEVICE)
            trigger_labels = trigger_labels.to(DEVICE)

            images = torch.cat((images, trigger_images), dim=0)
            labels = torch.cat((labels, trigger_labels), dim=0)

        # Clean optimizer gradients, get the output, get the loss, update gradients, update parameters
        optimizer.zero_grad()
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_description(f"Epoch [{epoch + 1}/30] Loss: {loss.item():.4f} Acc.: {100.0 * correct / total:.2f}%")

    # epoch loss and accuracy
    train_acc = 100.0 * correct / total
    avg_loss = total_loss / (batch_idx + 1)

    # epoch evaluation
    test_acc = evaluate(model, test_dataloader)
    trigger_acc = evaluate(model, trigger_dataloader)

    print(f"Epoch [{epoch + 1}/30]\tTrain Acc.: {train_acc:.2f}%\tTest-set Acc.: {test_acc:.2f}%\tTrigger-set Acc.: {trigger_acc:.2f}%\tAvg Loss: {avg_loss:.4f}")

    # save model
    if (epoch + 1) % 5 == 0:
        save_checkpoint(epoch + 1, model, optimizer, "./checkpoint")

# final save
save_checkpoint(30, model, optimizer, "./checkpoint/model.pt")


100%|██████████| 170498071/170498071 [00:10<00:00, 15866714.53it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Epoch [1/30] Loss: 1.4666 Acc.: 29.67%: 100%|██████████| 719/719 [11:28<00:00,  1.04it/s]


Epoch [1/30]	Train Acc.: 29.67%	Test-set Acc.: 38.83%	Trigger-set Acc.: 10.00%	Avg Loss: 1.8967


Epoch [2/30] Loss: 1.2952 Acc.: 53.41%: 100%|██████████| 719/719 [11:22<00:00,  1.05it/s]


...

Epoch [29/30] Loss: 0.5250 Acc.: 85.90%: 100%|██████████| 719/719 [11:34<00:00,  1.04it/s]


Epoch [29/30]	Train Acc.: 85.90%	Test-set Acc.: 74.12%	Trigger-set Acc.: 10.00%	Avg Loss: 0.4089


Epoch [30/30] Loss: 0.4561 Acc.: 86.13%: 100%|██████████| 719/719 [11:35<00:00,  1.03it/s]


Epoch [30/30]	Train Acc.: 86.13%	Test-set Acc.: 76.17%	Trigger-set Acc.: 6.00%	Avg Loss: 0.4041


RuntimeError: Parent directory ./checkpoint/model.pt does not exist.